# Portfolio Yml

In [1]:
import intake
import pandas as pd
import yaml
from yaml import SafeDumper

In [2]:
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

In [3]:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

In [4]:
catalog = intake.open_catalog("./project_prioritization/*.yml")

In [5]:
# Read original df
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [6]:
# Fill in empty county names with various
df_statewide["full_county_name"] = df_statewide["full_county_name"].fillna("Various")

In [7]:
# Filter for unique full county names
"""
toc = (
    df_statewide[["full_county_name"]]
    .drop_duplicates()
    .sort_values("full_county_name")
    .reset_index(drop=True)
)
"""

'\ntoc = (\n    df_statewide[["full_county_name"]]\n    .drop_duplicates()\n    .sort_values("full_county_name")\n    .reset_index(drop=True)\n)\n'

In [8]:
# Add quotes to every full county name
# https://stackoverflow.com/questions/41960346/how-to-add-quotes-to-every-element-in-non-numeric-column-of-a-dataframe-save
# df_statewide.update(df_statewide[["full_county_name"]].applymap('"{}"'.format))

In [9]:
# Create a sorted, unique list of all the counties in dataset
# parameter_counties = df_statewide.full_county_name.sort_values().unique().tolist()

In [10]:
# Open my test yml as called "test" and load it as "analyses_data"
with open("../project_prioritization/test.yml") as test:
    analyses_data = yaml.load(test, yaml.Loader)

In [11]:
# Empty chapter list
chapters_list = []

In [12]:
# For every county in my list of counties, loop through and create
# chapters and parameters
for county in (df_statewide.full_county_name.sort_values().unique()):
    # Empty dictionary
    chapter_dict = {}
    chapter_dict["params"] = {
        "parameter_county": county,
        "county_title": f'{county} County'}
    chapters_list += [chapter_dict]

In [13]:
parts_list = [{"chapters": chapters_list}]

# doesn't work for me
# analyses_data['parts'] = parts_list


In [14]:
# yaml.dump(parts_list) produces the document as a UTF-8 encoded str object.
# https://pyyaml.org/wiki/PyYAMLDocumentation
output = yaml.dump(parts_list)

In [15]:
with open("../project_prioritization/test.yml", "w") as test:
    test.write(output)